In [7]:
pip install openai

Note: you may need to restart the kernel to use updated packages.


In [8]:
pip install --upgrade openai

Note: you may need to restart the kernel to use updated packages.


In [34]:
import os
import openai
client = OpenAI(api_key="sk-proj-ilIgm1f785Qs1vCRSOzwT3BlbkFJIYwPOsEEvkCwSDBefrpq")

response = client.chat.completions.create(
  model="gpt-4-turbo",
  messages=[
    {"role": "system", "content": "You are a friendly assistant helping user to organize schedules."},
    {"role": "user", "content": "Can you remind me to pick up my dry cleaning tomorrow?"}
  ],
)

print(response.choices[0].message.content)

I can't set reminders directly, but I can suggest setting a reminder on your phone or calendar app for picking up your dry cleaning tomorrow. Would you like advice on how to set it up?


In [28]:

  
assistant = client.beta.assistants.create(
  name="Math Tutor",
  instructions="You are a personal math tutor. Write and run code to answer math questions.",
  tools=[{"type": "code_interpreter"}],
  model="gpt-4-turbo",
)

In [29]:
thread = client.beta.threads.create()

In [30]:
message = client.beta.threads.messages.create(
  thread_id=thread.id,
  role="user",
  content="I need to solve the equation `3x + 11 = 14`. Can you help me?"
)

In [31]:
from typing_extensions import override
from openai import AssistantEventHandler
 
# First, we create a EventHandler class to define
# how we want to handle the events in the response stream.
 
class EventHandler(AssistantEventHandler):    
  @override
  def on_text_created(self, text) -> None:
    print(f"\nassistant > ", end="", flush=True)
      
  @override
  def on_text_delta(self, delta, snapshot):
    print(delta.value, end="", flush=True)
      
  def on_tool_call_created(self, tool_call):
    print(f"\nassistant > {tool_call.type}\n", flush=True)
  
  def on_tool_call_delta(self, delta, snapshot):
    if delta.type == 'code_interpreter':
      if delta.code_interpreter.input:
        print(delta.code_interpreter.input, end="", flush=True)
      if delta.code_interpreter.outputs:
        print(f"\n\noutput >", flush=True)
        for output in delta.code_interpreter.outputs:
          if output.type == "logs":
            print(f"\n{output.logs}", flush=True)
 
# Then, we use the `stream` SDK helper 
# with the `EventHandler` class to create the Run 
# and stream the response.
 
with client.beta.threads.runs.stream(
  thread_id=thread.id,
  assistant_id=assistant.id,
  instructions="Please address the user as Jane Doe. The user has a premium account.",
  event_handler=EventHandler(),
) as stream:
  stream.until_done()


assistant > Sure, Jane Doe! Let's solve the equation \(3x + 11 = 14\).

We need to find the value of \(x\) that satisfies this equation. We start by isolating \(x\) on one side of the equation. Let's do the math.
assistant > code_interpreter

from sympy import symbols, Eq, solve

x = symbols('x')
equation = Eq(3*x + 11, 14)
solution = solve(equation)
solution

output >

[1]

assistant > The solution to the equation \(3x + 11 = 14\) is \(x = 1\).